# Artificial Intelligence for Games
- [Udemy Course](https://derivco.udemy.com/course/artificial-intelligence-for-simple-games/)

### Environment Setup

In [15]:
%run ../../../utilities/reinforcement_learning/environments/crypto_pair_trading_environment.ipynb

In [16]:
import pickle

In [17]:
model_file_path = '../../market_trading/data/featurized_market_data.p' #'./data/featurized_market_data.p' 

with open(model_file_path, 'rb') as fp:
    featurized_market_data = pickle.load(fp)

In [18]:
pair_name = 'AAVE_BTC'
pair_data = featurized_market_data[pair_name]

# This dataset contains open, close, high, low, volume and any numerical features / indicators you like.
price_movement_df = pair_data

In [19]:
env = CryptoPairTradingEnv(pair_data, pair_name)

### Genetic Algorithms

In [20]:
import numpy as np